# 多GPU训练

In [1]:
%matplotlib inline
import torch 
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l


简单网络

In [2]:
scale = 0.01
# 四个blocks,分别分给四块GPU
W1 = torch.randn(size=(20, 1, 3, 3)) * scale
b1 = torch.zeros(20)
W2 = torch.randn(size=(50, 20, 5, 5)) * scale
b2 = torch.zeros(50)
W3 = torch.randn(size=(800, 128)) * scale
b3 = torch.zeros(128)
W4 = torch.randn(size=(128, 10)) * scale
b4 = torch.zeros(10)
params = [W1, b1, W2, b2, W3, b3, W4, b4]

In [3]:
def lenet(X, params):
    h1_conv = F.conv2d(input=X, weight=params[0], bias=params[1])
    h1_activation = F.relu(h1_conv)
    h1 = F.avg_pool2d(input=h1_activation, kernel_size=(2, 2), stride=(2, 2))
    h2_conv = F.conv2d(input=h1, weight=params[2], bias=params[3])
    h2_activation = F.relu(h2_conv)
    h2 = F.avg_pool2d(input=h2_activation, kernel_size=(2, 2), stride=(2, 2))
    #??
    h2 = h2.reshape(h2.shape[0], -1)
    #?? mm
    h3_linear = torch.mm(h2, params[4]) + params[5]
    h3 = F.relu(h3_linear)
    y_hat = torch.mm(h3, params[6]) + params[7]
    return y_hat

loss = nn.CrossEntropyLoss(reduction='none')

向多个设备分发参数

In [4]:
# 把参数复制到gpu上
def get_params(params,device):
    #if p 已经在GPU了，就不会copy
    # 保险一点就是clone最好
    new_params = [p.clone().to(device) for p in params]
    for p in new_params:
        #需要计算梯度
        p.requires_grad()
    return new_params

# new_params = get_params(params, d2l.try_gpu(0))
# print('b1 weight:', new_params[1])
# print('b1 grad:', new_params[1].grad)

allreduce 函数将所有向量相加，并将结果广播给所有 GPU


![](imgs/2022-09-20-16-38-48.png)

In [5]:
def allreduce(data):
    for i in range(1, len(data)):
        #把其他GPU的向量加到cuda0
        data[0][:] += data[i].to(data[0].device)
    for i in range(1, len(data)):
        #在将cuda0加和后的res 广播到其他cuda
        data[i] = data[0].to(data[i].device)

# data = [torch.ones((1, 2), device=d2l.try_gpu(i)) * (i + 1) for i in range(2)]
# print('before allreduce:\n', data[0], '\n', data[1])
# allreduce(data)
# print('after allreduce:\n', data[0], '\n', data[1])

before allreduce:<br/>
 tensor([[1., 1.]], device='cuda:0') <br/>
 tensor([[2., 2.]], device='cuda:1')<br/>
after allreduce:<br/>
 tensor([[3., 3.]], device='cuda:0') <br/>
 tensor([[3., 3.]], device='cuda:1')<br/>

将一个小批量数据均匀地分布在多个 GPU 上

In [6]:
# data = torch.arange(20).reshape(4, 5)
# devices = [torch.device('cuda:0'), torch.device('cuda:1')]
# #有一个data，scatter可以均匀得把data划分到不同的cuda
# split = nn.parallel.scatter(data, devices)
# print('input :', data)
# print('load into', devices)
# print('output:', split)

input : tensor([[ 0,  1,  2,  3,  4],<br/>
        [ 5,  6,  7,  8,  9],<br/>
        [10, 11, 12, 13, 14],<br/>
        [15, 16, 17, 18, 19]])<br/>
load into [device(type='cuda', index=0), device(type='cuda', index=1)]<br/>
output: (tensor([[0, 1, 2, 3, 4],<br/>
        [5, 6, 7, 8, 9]], device='cuda:0'), tensor([[10, 11, 12, 13, 14],<br/>
        [15, 16, 17, 18, 19]], device='cuda:1'))<br/>

In [7]:
def split_batch(X, y, devices):
    """将`X`和`y`拆分到多个设备上"""
    #样本数相同
    assert X.shape[0] == y.shape[0]
    return (nn.parallel.scatter(X, devices), nn.parallel.scatter(y, devices))

在一个小批量上实现多 GPU 训练

In [8]:
def train_batch(X, y, device_params, devices, lr):
    #把X,y均匀划分，并且把他们放到不同的cuda
    X_shards, y_shards = split_batch(X, y, devices)
    # loss：计算每个cuda分到的批量的loss
    ls = [loss(lenet(X_shard, device_W),y_shard).sum() 
             for X_shard, y_shard, device_W in zip(
                 X_shards, y_shards, device_params)]
    # 计算梯度每个cuda上的梯度
    for l in ls:
        l.backward()
    #关闭梯度
    with torch.no_grad():
        for i in range(len(device_params[0])):
            #每一层：把各个cuda计算得到的梯度累加
            allreduce([device_params[c][i].grad for c in range(len(devices))])
    # 每个cuda各自更新参数
    for param in device_params:
        d2l.sgd(param, lr, X.shape[0])

定义训练函数

In [9]:
def train(num_gpus, batch_size, lr):
    train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
    devices = [d2l.try_gpu(i) for i in range(num_gpus)]
    #给每个cuda 复制参数
    device_params = [get_params(params, d) for d in devices]
    num_epochs = 10
    animator = d2l.Animator('epoch', 'test acc', xlim=[1, num_epochs])
    timer = d2l.Timer()
    for epoch in range(num_epochs):
        timer.start()
        for X, y in train_iter:
            train_batch(X, y, device_params, devices, lr)
            # 加入同步机制，保证每个cuda都做完了
            torch.cuda.synchronize()
        timer.stop()
        animator.add(epoch + 1, (d2l.evaluate_accuracy_gpu(
            lambda x: lenet(x, device_params[0]), test_iter, devices[0]),))
    print(f'test acc: {animator.Y[0][-1]:.2f}, {timer.avg():.1f} sec/epoch '
          f'on {str(devices)}')

在单个GPU上运行

In [10]:
train(num_gpus=1, batch_size=256, lr=0.2)

TypeError: 'bool' object is not callable

增加为2个GPU

In [ ]:
# train(num_gpus=2, batch_size=256, lr=0.2)

 实验结果显示单个gpu 跑 和两个gpu跑处理一个epoch的速度都相同
 - 通常是增加了gpu但是没有增加batch_size，batch均分给各个gpu后，变小了，做计算的矩阵会变小，影响性能，没用用满gpu
 - 通常保证，增加gpu后，各个gpu拿到与之前同样的batch_size,并且lr增大一些(x1.5)
 - 老师的example 在多gpu上run lenet ，测试精度比单GPU 降了一些(0.82 --> 0.64)，并且处理的速度也没有达到原来的两倍
    - 裸写的code，对pytorch框架不友好
    - lenet这个网络太小了，无法发挥多cpu的效果

![](imgs/2022-09-20-18-08-33.png)

![](imgs/2022-09-20-18-08-45.png)